# Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly as py
from matplotlib import pyplot as plt

# Import the data

In [2]:
column_names = ["id","dia","radius","texture","perimiter","area","smoothness","compactness","concavity","concanve points","symmetry","fractal dimension","SE radius","SE texture","SE perimeter","SE area","SE smoothness","SE compactness","SE concavity","SE concave points", "SE symmetry", "SE fractal dimension", "max radius","max texture","max perimeter","max area", "max smoothness","max compactness","max concavity","max concave points","max symmetry","max fractal dimension"]
data = pd.read_csv('wdbc.data',names=column_names)
data.head()

,id,dia,radius,texture,perimiter,area,smoothness,compactness,concavity,concanve points,...,max radius,max texture,max perimeter,max area,max smoothness,max compactness,max concavity,max concave points,max symmetry,max fractal dimension
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# Preparing the Data

###### Creation of an array of correct labels

In [3]:
correct_labels = np.array(data["dia"].apply(lambda x:1 if x=="M" else 0))
correct_labels_inverted = np.abs(correct_labels-1)
correct_labels = np.vstack((correct_labels, correct_labels_inverted)).T

###### Normalisation of the data

In [4]:
df = data.iloc[:,2:].copy()

col = 0
while col <df.shape[1]:
    df.iloc[:,col] = ((df.iloc[:,col])-df.iloc[:,col].min())/(df.iloc[:,col].max()-df.iloc[:,col].min())
    col+=1

In [5]:
df.head()

,radius,texture,perimiter,area,smoothness,compactness,concavity,concanve points,symmetry,fractal dimension,...,max radius,max texture,max perimeter,max area,max smoothness,max compactness,max concavity,max concave points,max symmetry,max fractal dimension
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595


In [6]:
##from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=42)

In [7]:
input_layer = df.shape[1]
hidden_layer1 = 10
hidden_layer2 = 10
output_layer = 2

weights_InputToHidden1 = np.array(np.random.uniform(-0.5,0.5,(input_layer, hidden_layer1)))
weights_Hidden1ToHidden2 = np.random.uniform(-0.5,0.5,(hidden_layer1, hidden_layer2))
weights_Hidden2ToOutput = np.random.uniform(-0.5,0.5,(hidden_layer2, output_layer))
bias_InputToHidden1 =np.zeros((1,hidden_layer1))
bias_Hidden1ToHidden2 =np.zeros((1,hidden_layer2))
bias_Hidden2ToOutput = np.zeros((1,output_layer))

alpha = 0.002
correct_diagnosis = 0
repeats = 200

# Relevant Functions

### Sigmoid Function  
### $f(x) = \frac{1}{1+e^x}$

In [8]:
def sigmoid(X):
    return 1/(1 + np.exp(-X))

def deriv_sigmoid(X):
    return X*(1-X)

def ReLu(X):
    return np.maximum(0,X)

def deriv_ReLu(Z):
    return Z >0

def tanh(X):
    return np.tanh(X)

def deriv_tanh(X):
    return 1-np.power(X,2)

def Forward_Propagate(weights_InputToHidden1,bias_InputToHidden1,weights_Hidden1ToHidden2,bias_Hidden1ToHidden2,weights_Hidden2ToOutput,bias_Hidden2ToOutput,inputs):
    hidden1_unscaled = inputs @ weights_InputToHidden1 + bias_InputToHidden1
    #tanh(x) activation
    hidden1 = tanh(hidden1_unscaled)
    hidden2_unscaled = hidden1 @ weights_Hidden1ToHidden2 + bias_Hidden1ToHidden2
    #ReLu(x) activation
    hidden2 = ReLu(hidden2_unscaled)
    outputs_unscaled = hidden2 @ weights_Hidden2ToOutput + bias_Hidden2ToOutput
    #Sigmoid(x) activation
    outputs = sigmoid(outputs_unscaled)
    return hidden1_unscaled,hidden1,hidden2_unscaled,hidden2,outputs_unscaled,outputs

def Cost_function(output_layer,outputs):
    error = 1/output_layer*np.sum((outputs-1)**2,axis=0)
    return error

def Backward_Propagation(weights_InputToHidden1,bias_InputToHidden1,weights_Hidden1ToHidden2,bias_Hidden1ToHidden2,weights_Hidden2ToOutput,bias_Hidden2ToOutput,outputs,hidden1,hidden2,alpha):
    delta_outputs = outputs - correct_labels[row]
    weights_Hidden2ToOutput += -alpha * delta_outputs * np.transpose(hidden2)
    bias_Hidden2ToOutput += -alpha * delta_outputs
    delta_hidden2 = weights_Hidden2ToOutput @ np.transpose(delta_outputs) * deriv_ReLu(np.transpose(hidden2))
    weights_Hidden1ToHidden2 += np.transpose(-alpha * delta_hidden2 @ hidden1)
    bias_Hidden1ToHidden2 += np.transpose(-alpha * delta_hidden2)
    
    delta_hidden1 = weights_Hidden1ToHidden2 @ delta_hidden2 * deriv_tanh(np.transpose(hidden1))
    
    weights_InputToHidden1 += np.transpose(-alpha * delta_hidden1 @ inputs)
    bias_InputToHidden1 += np.transpose(-alpha * delta_hidden1)
    return delta_outputs, weights_Hidden2ToOutput,bias_Hidden2ToOutput,delta_hidden2,weights_Hidden1ToHidden2,bias_Hidden1ToHidden2,delta_hidden1,weights_InputToHidden1,bias_InputToHidden1

In [9]:
cost_list = []
epoch_list = []

for repeat in range(repeats):
    for row in range(df.shape[0]):
        inputs = np.array(df.iloc[row])
        inputs.shape += (1,)
        inputs = np.transpose(inputs)
        
        hidden1_unscaled,hidden1,hidden2_unscaled,hidden2,outputs_unscaled,outputs = Forward_Propagate(weights_InputToHidden1,bias_InputToHidden1,weights_Hidden1ToHidden2,bias_Hidden1ToHidden2,weights_Hidden2ToOutput,bias_Hidden2ToOutput,inputs)
        error = Cost_function(output_layer,outputs)
        
        correct_diagnosis += int(np.argmax(outputs) == np.argmax(correct_labels[row]))
        
        delta_outputs,weights_Hidden2ToOutput,bias_Hidden2ToOutput,delta_hidden2,weights_Hidden1ToHidden2,bias_Hidden1ToHidden2,delta_hidden1,weights_InputToHidden1,bias_InputToHidden1 = Backward_Propagation(weights_InputToHidden1,bias_InputToHidden1,weights_Hidden1ToHidden2,bias_Hidden1ToHidden2,weights_Hidden2ToOutput,bias_Hidden2ToOutput,outputs,hidden1,hidden2,alpha)
        
    print(f"Accuracy: {round((correct_diagnosis/569)*100,2)}%")
    correct_diagnosis = 0
    
    if repeat %10==0:
        cost_list.append(error)
        epoch_list.append(repeat)


Accuracy: 62.92%
Accuracy: 65.03%
Accuracy: 73.99%
Accuracy: 84.71%
Accuracy: 91.21%
Accuracy: 92.97%
Accuracy: 94.38%
Accuracy: 94.9%
Accuracy: 95.25%
Accuracy: 95.08%
Accuracy: 95.08%
Accuracy: 95.43%
Accuracy: 95.96%
Accuracy: 96.13%
Accuracy: 96.31%
Accuracy: 95.96%
Accuracy: 95.96%
Accuracy: 95.96%
Accuracy: 95.96%
Accuracy: 96.13%
Accuracy: 96.49%
Accuracy: 96.66%
Accuracy: 97.01%
Accuracy: 97.01%
Accuracy: 97.01%
Accuracy: 97.01%
Accuracy: 97.19%
Accuracy: 97.19%
Accuracy: 97.36%
Accuracy: 97.36%
Accuracy: 97.36%
Accuracy: 97.36%
Accuracy: 97.36%
Accuracy: 97.36%
Accuracy: 97.54%
Accuracy: 97.54%
Accuracy: 97.54%
Accuracy: 97.54%
Accuracy: 97.54%
Accuracy: 97.72%
Accuracy: 97.72%
Accuracy: 97.72%
Accuracy: 97.89%
Accuracy: 97.89%
Accuracy: 97.89%
Accuracy: 97.89%
Accuracy: 97.89%
Accuracy: 97.89%
Accuracy: 97.89%
Accuracy: 98.07%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24%
Accuracy: 98.24